In [ ]:
from __future__ import print_function
import numpy as np
import argparse
import torch
import torch.nn.functional as F
from optimizer import PruneAdam
from model import LeNet, AlexNet
from utils import regularized_nll_loss, admm_loss, \
    initialize_Z_and_U, update_X, update_Z, update_Z_l1, update_U, \
    print_convergence, print_prune, apply_prune, apply_l1_prune
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models
from tqdm import tqdm
from Fed import FedAvg
import copy
import os
from PIL import Image

# from torch.utils.tensorboard import SummaryWriter
# %load_ext tensorboard

In [ ]:
class CustomDataSet(Dataset):
    def __init__(self, main_dir, x, y, transform):
        self.main_dir = main_dir
        self.transform = transform
        self.y = y
        self.all_imgs = x
#         self.total_imgs = natsort.natsorted(all_imgs)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.all_imgs[idx])
        image = Image.open(img_loc).convert("RGB")
        tensor_image = self.transform(image)
        return tensor_image,self.y[idx]

In [ ]:
# Training settings
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--dataset', type=str, default="cifar10", choices=["mnist", "cifar10"],
                    metavar='D', help='training dataset (mnist or cifar10)')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--percent', type=list, default=[0.8, 0.92, 0.991, 0.93],
                    metavar='P', help='pruning percentage (default: 0.8)')
parser.add_argument('--alpha', type=float, default=5e-4, metavar='L',
                    help='l2 norm weight (default: 5e-4)')
parser.add_argument('--rho', type=float, default=1e-2, metavar='R',
                    help='cardinality weight (default: 1e-2)')
parser.add_argument('--l1', default=False, action='store_true',
                    help='prune weights with l1 regularization instead of cardinality')
parser.add_argument('--l2', default=False, action='store_true',
                    help='apply l2 regularization')
parser.add_argument('--num_pre_epochs', type=int, default=3, metavar='P',
                    help='number of epochs to pretrain (default: 3)')
parser.add_argument('--num_epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--num_re_epochs', type=int, default=3, metavar='R',
                    help='number of epochs to retrain (default: 3)')
parser.add_argument('--lr', type=float, default=1e-3, metavar='LR',
                    help='learning rate (default: 1e-2)')
parser.add_argument('--adam_epsilon', type=float, default=1e-8, metavar='E',
                    help='adam epsilon (default: 1e-8)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--save-model', action='store_true', default=False,
                    help='For Saving the current Model')
parser.add_argument('--num_users', action='store_true', default=1,
                    help='Number of users in network')
parser.add_argument('--model', action='store_true', default='resnet',
                    help='Model to train')
args = parser.parse_args(args=[])

In [ ]:
classDict = {'not_smiling':0, 'smiling':1}

# Define a function to separate CIFAR classes by class index

def get_class_i(x, y, i):
    """
    x: trainset.train_data or testset.test_data
    y: trainset.train_labels or testset.test_labels
    i: class label, a number between 0 to 9
    return: x_i
    """
    # Convert to a numpy array
    y = np.array(y)
    # Locate position of labels that equal to i
    pos_i = np.argwhere(y == i)
    # Convert the result into a 1-D list
    pos_i = list(pos_i[:,0])
    # Collect all data that match the desired label
#     x_i = [x[j] for j in pos_i]
    
    return pos_i

In [5]:
# Standard transformations for improving celebA. 
# Transformations A
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop((218,178), padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Transformations B
RC   = transforms.RandomCrop((218,178), padding=4)
RHF  = transforms.RandomHorizontalFlip()
RVF  = transforms.RandomVerticalFlip()
NRM  = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
TT   = transforms.ToTensor()
TPIL = transforms.ToPILImage()

# Transforms object for trainset with augmentation
transform_with_aug = transforms.Compose([RC, RHF, TT, NRM])
# Transforms object for testset with NO augmentation
transform_no_aug   = transforms.Compose([TT, NRM])

# Downloading/Louding CELEBA data
trainset = torchvision.datasets.CelebA(root='../../../data/celebA', split = 'train',
                                        download=True, transform=transform_with_aug)

testset = torchvision.datasets.CelebA(root='../../../data/celebA', split='test',
                                       download=True, transform=transform_no_aug)

classDict = {'not_smiling':0, 'smiling':1}

# Separating trainset/testset data/label
x_train  = trainset
x_test   = testset
y_train  = trainset.attr[:,31] ## 31 is smile
y_test   = testset.attr[:,31] 

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [6]:
def create_dict(x_train=x_train, x_test=x_test, y_train=y_train, y_test=y_test):
    # If we are saving a fraction of random data to be used in training
    frac1 = int(len(x_test) * 0.316666666667)
    frac2 = int(len(x_test) * 0.633333333333)

    x_train1 = list(range(0,frac1))
    x_train2 = list(range(frac1,frac2))
    x_train3 = list(range(frac2,len(x_test)))
    
#     trainset1 = CustomDataSet('../../../data/celebA/celeba/img_align_celeba',x=x_train1,y=y_train1, transform=transform_with_aug)
#     trainset2 = CustomDataSet('../../../data/celebA/celeba/img_align_celeba',x=x_train2,y=y_train2, transform=transform_with_aug)
#     trainset3 = CustomDataSet('../../../data/celebA/celeba/img_align_celeba',x=x_train3,y=y_train3, transform=transform_with_aug)
#     testset = CustomDataSet('../../../data/celebA/celeba/img_align_celeba',x=x_test,y=y_test, transform=transform_no_aug)

    
    dict_users = {0: x_train3, 1:x_train1, 2:x_train2}
    return dict_users

class DatasetSplit(torch.utils.data.Dataset):
    def __init__(self, dataset, idxs):
        self.dataset = dataset
        self.idxs = list(idxs)

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        image, label = self.dataset[self.idxs[item]]
        return image, label

In [7]:
gossip = []

def pretrain(args, model, device, train_loader, test_loader, optimizer):
    for epoch in range(args.num_pre_epochs):
        print('Pre epoch: {}'.format(epoch + 1))
        model.train()
        for batch_idx, (data, target) in enumerate(tqdm(train_loader)):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = regularized_nll_loss(args, model, output, target)
            loss.backward()
            optimizer.step()
        test(args, model, device, test_loader)


def train(args, model, device, train_loader, test_loader, optimizer, Z, U, report=False):
    model.train()
    print('Epoch: {}'.format(epoch + 1))
    for batch_idx, (data, target) in enumerate(tqdm(train_loader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = admm_loss(args, device, model, Z, U, output, target)
        loss.backward()
        optimizer.step()
    X = update_X(model)
    Z = update_Z_l1(X, U, args) if args.l1 else update_Z(X, U, args)
    U = update_U(U, X, Z)
    print_convergence(model, X, Z)
    test(args, model, device, test_loader, report)


iter = 0
def test(args, model, device, test_loader, report=False):
    model.eval()
    test_loss = 0
    correct = 0
    global iter
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    gossip.append(correct / len(test_loader.dataset))
    
    if report:
#         writer.add_scalar('train/loss_gossip_admm4', test_loss, iter)
#         writer.add_scalar('valid/accuracy_gossip_admm4', correct / len(test_loader.dataset), iter)
        iter+=1


def retrain(args, model, mask, device, train_loader, test_loader, optimizer):
    for epoch in range(args.num_re_epochs):
        print('Re epoch: {}'.format(epoch + 1))
        model.train()
        for batch_idx, (data, target) in enumerate(tqdm(train_loader)):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.prune_step(mask)

        test(args, model, device, test_loader)

In [8]:
### MAIN

what_gpu = 2
torch.manual_seed(args.seed)
device = torch.device('cuda:{}'.format(what_gpu) if torch.cuda.is_available() else 'cpu')
kwargs = {'num_workers': 5, 'pin_memory': True}

args.percent = [0.6, 0.7, 0.8, 0.94, 0.95, 0.99, 0.99, 0.93]
args.num_pre_epochs = 1
args.num_epochs = 60
args.num_re_epochs = 5
args.num_users = 3
args.dataset = 'cifar10'
args.model = 'resnet'
args.l1 = True
args.l2 = False

trainset=CustomDataSet('../../../data/celebA/celeba/img_align_celeba',x=x_train.filename,y=y_train, transform=transform_with_aug)
testset=CustomDataSet('../../../data/celebA/celeba/img_align_celeba',x=x_test.filename,y=y_test, transform=transform_no_aug)
test_loader = torch.utils.data.DataLoader(testset , batch_size=args.batch_size, shuffle=True, **kwargs)
dict_users = create_dict()

In [9]:
mdlz = dict()
for usr in range(args.num_users):
    train_loader = torch.utils.data.DataLoader(DatasetSplit(trainset, dict_users[usr]), batch_size=args.batch_size, shuffle=True, **kwargs)
    if args.model == 'resnet':
        model = models.resnet50(pretrained=True)
        num_ftrs = model.fc.in_features
        model.fc = torch.nn.Linear(num_ftrs, 2)
        model.to(device)
    else:
        model = LeNet().to(device) if args.dataset == "mnist" else AlexNet().to(device)
    
    optimizer = PruneAdam(model.named_parameters(), lr=args.lr, eps=args.adam_epsilon)
    pretrain(args, model, device, train_loader, test_loader, optimizer)
    Z, U = initialize_Z_and_U(model)
    mdlz[usr] = (model, optimizer, Z, U)
    
for epoch in range(args.num_epochs):
    w_locals=[]
    idxs_users = np.random.choice(range(args.num_users), 2, replace=False)
    for usr in idxs_users:
        report = True if usr == 0 else False
        train_loader = torch.utils.data.DataLoader(DatasetSplit(trainset, dict_users[usr]), batch_size=args.batch_size, shuffle=True, **kwargs)
        train(args, mdlz[usr][0], device, train_loader, test_loader, mdlz[usr][1], mdlz[usr][2], mdlz[usr][3], report=report)
        w = mdlz[usr][0].state_dict()
        w_locals.append(copy.deepcopy(w))

    # update global weights
    w_glob = FedAvg(w_locals)

    for idx in idxs_users:
        # copy weight to net_glob
        mdlz[idx][0].load_state_dict(w_glob)

  0%|          | 0/115 [00:00<?, ?it/s]

Pre epoch: 1


100%|██████████| 115/115 [00:25<00:00,  4.53it/s]



Test set: Average loss: -365.3359, Accuracy: 17946/19962 (90%)



  0%|          | 0/99 [00:00<?, ?it/s]

...

normalized norm of (weight - projection)
(conv1.weight): 0.3118
(bn1.weight): 0.1800
(layer1.0.conv1.weight): 0.4517
(layer1.0.bn1.weight): 0.2287
(layer1.0.conv2.weight): 0.7401
(layer1.0.bn2.weight): 0.2825
(layer1.0.conv3.weight): 0.6837
(layer1.0.bn3.weight): 0.2635
(layer1.0.downsample.0.weight): 0.5183
(layer1.0.downsample.1.weight): 0.1912
(layer1.1.conv1.weight): 0.7497
(layer1.1.bn1.weight): 0.2422
(layer1.1.conv2.weight): 0.7789
(layer1.1.bn2.weight): 0.2653
(layer1.1.conv3.weight): 0.7288
(layer1.1.bn3.weight): 0.4281
(layer1.2.conv1.weight): 0.7622
(layer1.2.bn1.weight): 0.2733
(layer1.2.conv2.weight): 0.7779
(layer1.2.bn2.weight): 0.2247
(layer1.2.conv3.weight): 0.7401
(layer1.2.bn3.weight): 0.3506
(layer2.0.conv1.weight): 0.7243
(layer2.0.bn1.weight): 0.2429
(layer2.0.conv2.weight): 0.8596
(layer2.0.bn2.weight): 0.2483
(layer2.0.conv3.weight): 0.7630
(layer2.0.bn3.weight): 0.3078
(layer2.0.downsample.0.weight): 0.7893
(layer2.0.downsample.1.weight): 0.2757
(layer2.1.conv1

  0%|          | 0/115 [00:00<?, ?it/s]


Test set: Average loss: -10465.6272, Accuracy: 9975/19962 (50%)

Epoch: 8


 53%|█████▎    | 61/115 [00:22<00:20,  2.66it/s]


KeyboardInterrupt: 

In [ ]:
### Apply pruning
for usr in range(args.num_users):
    mask = apply_l1_prune(mdlz[usr][0], device, args) if args.l1 else apply_prune(mdlz[usr][0], device, args)
    print_prune(model)
    test(args, mdlz[usr][0], device, test_loader)
    retrain(args, model, mask, device, train_loader, test_loader, optimizer)

# Testing ground

In [ ]:
from torch.utils.data import DataLoader
import torch.nn.functional as F

confusion_matrix = torch.zeros(10, 10)
mdlz[0][0].eval()
# testing
test_loss = 0
correct = 0

l = len(test_loader)
for idx, (data, target) in enumerate(test_loader):
    data, target = data.to(device), target.to(device)
    log_probs = mdlz[0][0](data)
    # sum up batch loss
    test_loss += F.cross_entropy(log_probs, target, reduction='sum').item()
    # get the index of the max log-probability
    y_pred = log_probs.data.max(1, keepdim=True)[1]
    correct += y_pred.eq(target.data.view_as(y_pred)).long().cpu().sum()

    for t, p in zip(target.data, y_pred.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

In [ ]:
print(confusion_matrix)

In [ ]:
gossip